In [1]:
import pygame as pg
import pygame.draw as dw
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

pygame 2.6.0 (SDL 2.28.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


# RK4
Do tipo
$$\frac{dr}{dt} = f(x, t)$$

Para uma eq de movimento podemos ter $\frac{d^2\vec r}{dt^2}=f(\vec r,\frac{d\vec r}{dt}, t)$, que podemos resolver como:
$$\begin{cases}
\frac{d\vec r}{dt}=\vec v\\
\frac{d\vec v}{dt}= f(\vec r,\vec v,t)
\end{cases}$$

In [ ]:
# Aqui, a forma como se coloca r na função f() depende completamente de como f está definida e funciona

def RK4_2var(f, xi, yi, a, b, N):
    x, y = np.zeros(N), np.zeros(N)
    x[0] = xi
    y[0] = yi
    r = np.array([x,y]).T # transposta para cada linha ser um ponto (x,y)
    h = (b-a)/N
    t = np.linspace(a,b,N)
    for i in range(1, N):
        k1 = h * f(*r[i-1], t[i-1])
        k2 = h * f(*r[i-1] + 0.5*k1, t[i-1] + 0.5*h)
        k3 = h * f(*r[i-1] + 0.5*k2, t[i-1] + 0.5*h)
        k4 = h * f(*r[i-1] + k3, t[i-1] + h)

        r[i] = r[i-1] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
    
    return (r.T), t

# Pêndulo Simples
Deduzir:
$$\frac{d^2\theta}{dt^2} = -\frac{g}{L}\sin\theta$$
ou seja:
$$\begin{cases}\frac{d\theta}{dt} = \omega\\ \frac{d\omega}{dt}=-\frac{g}{L}\sin\theta\end{cases}$$
em que:

$$x=R\sin\theta~,~y=R\cos\theta~,~\omega=Rv$$

In [5]:
def animate():
    L = 1000
    disp = pg.display.set_mode((L,L), pg.RESIZABLE)
    pg.display.set_caption("Teste #1")
    clk = pg.time.Clock()
    FPS = 60

    # centro e raio de circulo
    C = np.array([L/2,L/2])
    dw.circle(disp, (255,255,255), C, 10, 0)

    # bola
    R = L/100
    r = C + np.array([L/4,0])
    comp = np.linalg.norm(r-C)
    th = np.arctan2((r-C)[0], (r-C)[1])
    v = np.array([0,-1])
    al = np.arctan2(v[0], v[1])
    phi = np.pi/2 - al - th
    om = np.linalg.norm(v) * np.cos(phi) / comp
    sol = np.array([th, om])


    colorTraj = [100,100,100]
    points = [(r[0],r[1])]

    m = 1 # massa (kg)
    
    N = 500
    dt = 100/N
    
    e = 0.5
    g = 10

    def f(th, om, t):
        dom = -g/comp * np.sin(th)
        return np.array([om, dom], float)
        

    for i in range(N):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                return None
            if event.type == pg.VIDEORESIZE:
                disp = pg.display.set_mode((event.w, event.h),pg.RESIZABLE)
                L = np.min([event.h, event.w])
                yL = 0.75*L
        disp.fill((0,0,0)) # refresh fundo da pagina

        dw.circle(disp, (255,255,255), C, 10, 0)
        
        t = i*dt
        k1 = dt * f(*sol, t)
        k2 = dt * f(*sol + 0.5*k1, t + 0.5*dt)
        k3 = dt * f(*sol + 0.5*k2, t + 0.5*dt)
        k4 = dt * f(*sol + k3, t + dt)

        sol = sol + 1/6*(k1 + 2*k2 + 2*k3 + k4)

        r = C.copy()
        r[0] += comp * np.sin(sol[0])
        r[1] += comp * np.cos(sol[0])

        points.append((r[0],r[1]))
        if i % 3 == 0: # 0,3,6,9 
            if colorTraj[0] + 2 > 255: colorTraj[0] = 0
            colorTraj[0] += 2
        if i-1 % 3 == 0: # 1,4,7,10
            if colorTraj[1] + 4 > 255: colorTraj[1] = 0
            colorTraj[1] += 4
        if i-2 % 3 == 0: # 2,5,8,11
            if colorTraj[2] - 2 < 0: colorTraj[2] = 255
            colorTraj[2] -= 2
        dw.lines(disp, colorTraj, False, points, 2)

        dw.circle(disp, (255,255,255), r, R, 0)
        dw.line(disp, (255,255,255), C, r)

        pg.display.update()
        clk.tick(FPS)
        x3 = pg.surfarray.pixels3d(disp)
        array = np.uint8(x3)
        im = Image.fromarray(array)
        im = im.save(f'./photos/{i:04d}.png', 'PNG')



animate()
pg.quit()

